dataframe with parameters

In [5]:
using DataFrames, CSV
df = CSV.read("hiv_pam.csv", DataFrame);
println("$df")

31×6 DataFrame
 Row │ definition                         symbol    unit                               value        value2         TexSymbols
     │ String                             String15  String                             Float64      Float64        String15
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Естественная смертность 𝑇_1        d1        1/день                                  0.02        0.02       $d_1$
   2 │ Интенсивность  вирусного лизиса …  delta     1/день                                  0.7         0.2095     $\\delta$
   3 │ естественная смертность 𝑇_2, 𝑇_2…  d2        1/день                                  0.005       0.003096   $d_2$
   4 │ Коэффициент потенциальной эффект…  f         [0,1]                                   0.34        0.5068     $f$
   5 │ Относительная эффективность инги…  epsilon2  [0,1]                                   0.0         0.0        $\\

symbolic mathematical model

In [2]:
using NeuralPDE, Lux, ModelingToolkit, Optimization, OptimizationOptimJL, OrdinaryDiffEq, Plots
import ModelingToolkit: Interval, infimum, supremum

In [1]:
using ModelingToolkit

In [2]:
ε_1, ε_2, a_A, a_T = 0.0, 0.0,0.0, 0.008
u = 0.0; rho_1 = 1.0; rho_2 = 1.0; 
@parameters t, d_1, k_1, γ_T, γ_E, δ_E2, p_T, λT,
            d_2, k_2, m, a_E, c, δ_E1, N_T, K_V,
            K_γ, λ_E, d_E, f, K_S, K_d, K_b2, K_b1, δ, p_E,
            b_E1, b_E2

28-element Vector{Num}:
    t
  d_1
  k_1
  γ_T
   γ_E
   δ_E2
  p_T
   λT
  d_2
  k_2
    m
   a_E
    c
    ⋮
  K_γ
   λ_E
   d_E
     f
  K_S
  K_d
 K_b2
 K_b1
    δ
   p_E
   b_E1
   b_E2

In [3]:
@variables T_1(..), Ts_1(..), T_2(..), Ts_2(..), V_1(..), V_2(..), E_1(..), E_2(..) 
Dt = Differential(t)

(::Differential) (generic function with 2 methods)

In [4]:


eqs=[
        Dt(T_1(t))~-d_1*T_1(t)-(1-u*ε_1)*k_1*V_1(t)*T_1(t)-γ_T*T_1(t)+p_T*(a_T*V_1(t))/(V_1(t)+K_V)*T_2(t)+p_T*a_A*T_2(t),
        Dt(Ts_1(t))~(1-u*ε_1)*k_1*V_1(t)*T_1(t)-δ*Ts_1(t)-m*E_1(t)*Ts_1(t)-γ_T*Ts_1(t)+p_T*(a_T*V_1(t))/(V_1(t)+K_V)*Ts_2(t)+p_T*a_A*Ts_2(t),
        Dt(T_2(t))~λT*K_S/(V_1(t)+K_S)+γ_T*T_1(t)-d_2*T_2(t)-(1-f*u*ε_1)*k_2*V_1(t)*T_2(t)-(a_T*V_1(t))/(V_1(t)+K_V)*T_2(t)-a_A*T_2(t),
        Dt(Ts_2(t))~γ_T*Ts_1(t)+(1-f*u*ε_1)*k_2*V_1(t)*T_2(t)-d_2*T_2(t)-(a_T*V_1(t))/(V_1(t)+K_V)*Ts_2(t)-a_A*Ts_2(t),
        Dt(V_1(t))~(1-u*ε_2)*1000.0*N_T*δ*Ts_1(t)-c*V_1(t)-1000.0*((1-u*ε_1)*rho_1*k_1*T_1(t)+(1-f*u*ε_1)*rho_2*k_2*T_2(t))*V_1(t),
        Dt(V_2(t))~u*ε_2*1000.0*N_T*δ*Ts_1(t)-c*V_2(t),
        Dt(E_1(t))~λ_E+b_E1*Ts_1(t)*E_1(t)/(Ts_1(t)+K_b1)-d_E*Ts_1(t)*E_1(t)/(Ts_1(t)+K_d)-δ_E1*E_1(t)-γ_E*(T_1(t)+Ts_1(t))*E_1(t)/(T_1(t)+Ts_1(t)+K_γ)+p_E*a_E*V_1(t)*E_2(t)/(V_1(t)+K_V),
        Dt(E_2(t))~γ_E*(T_1(t)+Ts_1(t))*E_1(t)/(T_1(t)+Ts_1(t)+K_γ)+b_E2*K_b2*E_2(t)/(E_2(t)+K_b2)-δ_E2*E_2(t)-a_E*V_1(t)*E_2(t)/(V_1(t)+K_V)
]




8-element Vector{Equation}:
 Differential(t)(T_1(t)) ~ (0.008p_T*T_2(t)*V_1(t)) / (K_V + V_1(t)) - d_1*T_1(t) - γ_T*T_1(t) - k_1*T_1(t)*V_1(t)
 Differential(t)(Ts_1(t)) ~ (0.008p_T*Ts_2(t)*V_1(t)) / (K_V + V_1(t)) + k_1*T_1(t)*V_1(t) - γ_T*Ts_1(t) - δ*Ts_1(t) - m*E_1(t)*Ts_1(t)
 Differential(t)(T_2(t)) ~ γ_T*T_1(t) + (K_S*λT) / (K_S + V_1(t)) + (-0.008T_2(t)*V_1(t)) / (K_V + V_1(t)) - d_2*T_2(t) - k_2*T_2(t)*V_1(t)
 Differential(t)(Ts_2(t)) ~ γ_T*Ts_1(t) + (-0.008Ts_2(t)*V_1(t)) / (K_V + V_1(t)) + k_2*T_2(t)*V_1(t) - d_2*T_2(t)
 Differential(t)(V_1(t)) ~ 1000.0N_T*δ*Ts_1(t) - c*V_1(t) - (1000.0k_1*T_1(t) + 1000.0k_2*T_2(t))*V_1(t)
 Differential(t)(V_2(t)) ~ -c*V_2(t)
 Differential(t)(E_1(t)) ~ λ_E + (b_E1*E_1(t)*Ts_1(t)) / (K_b1 + Ts_1(t)) + (-d_E*E_1(t)*Ts_1(t)) / (K_d + Ts_1(t)) + (a_E*p_E*E_2(t)*V_1(t)) / (K_V + V_1(t)) + (-γ_E*(T_1(t) + Ts_1(t))*E_1(t)) / (K_γ + T_1(t) + Ts_1(t)) - δ_E1*E_1(t)
 Differential(t)(E_2(t)) ~ (K_b2*b_E2*E_2(t)) / (K_b2 + E_2(t)) + (γ_E*(T_1(t) + Ts_1(t))

initial data

In [6]:
sim = [T_1(0),Ts_1(0),T_2(0),Ts_2(0),V_1(0),V_2(0),E_1(0),E_2(0)];
init_df = [1.0 for i in 1:8]

8-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [7]:
bcs = [s ~ i for (s,i) in zip(sim,init_df)]


8-element Vector{Equation}:
 T_1(0) ~ 1.0
 Ts_1(0) ~ 1.0
 T_2(0) ~ 1.0
 Ts_2(0) ~ 1.0
 V_1(0) ~ 1.0
 V_2(0) ~ 1.0
 E_1(0) ~ 1.0
 E_2(0) ~ 1.0

In [ ]:
domains = [t ∈ Interval(0.0,Final_Time)]
dt = 0.5

In [ ]:
function model_hiv(dX,X,p,t)

    rho1 = 1.; rho2 = 1.;
    ksi1 = p[11]*p[32]; ksi2 = p[5]*p[32];     
    
      dX[1] = -p[1] * X[1] - (1 - ksi1) * p[21] * X[5] * X[1] - p[14] * X[1] + p[18] * (p[7] *(X[5]/(X[5]+p[13]))+p[6]) * X[3];  #dT1
        
      dX[2] = (1. - ksi1) * p[21] * X[5] * X[1] - p[2] * X[2] - p[12] * X[7] * X[2] - p[14] * X[2] + p[18] * (p[7] * (X[5]/(X[5]+p[13])) + p[6]) * X[4]; #dT*_1
        
      dX[3] = p[23] * (p[24]/(X[5]+p[24])) + p[14] * X[1] - p[3] * X[3] - (1. - p[4] * ksi1)  * p[22] * X[5] * X[3] - (p[7] * (X[5]/(X[5]+p[13])) + p[6]) * X[3]; #dT2
      
      dX[4] = p[14] * X[2] + (1 - p[4] * ksi1)  * p[22] * X[5] * X[3] - p[3] * X[4] - (p[7] * (X[5]/(X[5]+p[13])) + p[6]) * X[4]; #dT_2
        
      dX[5] = (1. - ksi2) * 10^3 * p[25] * p[2] * X[2] - p[15] * X[5] - 10^3 * ((1 - ksi1) * rho1 * p[21] * X[1] * X[5]) - 10^3 * ((1. - p[4] * ksi1) * rho2 * p[22] * X[3] * X[5]) ; #dVI
        
      dX[6] = ksi2 * 10^3 * p[25] * p[2] * X[2] - p[15] * X[6]; #dVNI
        
      dX[7] = p[26] + p[31] * (X[2]/(X[2]+p[28])) * X[7] - p[16] * (X[2]/(X[2]+p[30])) * X[7] - p[17] * X[7] - p[9] * ((X[1]+X[2])/(X[1]+X[2]+p[27])) * X[7] + p[19] * p[8] * (X[5]/(X[5]+p[13])) * X[8]; #p[16]1
        
      dX[8] = p[9] * ((X[1]+X[2])/(X[1]+X[2]+p[27])) * X[7] + p[20] * (p[29]/(X[8]+p[29]))* X[8] - p[10] * X[8] - p[8] * (X[5]/(X[5]+p[13])) * X[8];
     
     
end

upload medical-experemental data for u_
- Virual Load data
- Total T-cells

In [6]:
using DataFrames, CSV
name_Time = ["Pre-infection",
    "0 Weeks",
    "1 Weeks",
    "2 Weeks",
    "3 Weeks",
    "4 Weeks",
    "6 Months",
    "1 Year"]
    
val_Time =[0.0,
    7.0,
    14.0,
    21.0,
    28.0,
    35.0,
    180.0,
    365.0]
#P1
val_VL = [0.0, 5.89, 7.96,6.3,5.91,5.87,4.5,5.0]
val_CD4 = [1200.,890., 600.,420.,550.,520.,560.,580.]

meas_data = DataFrame(name=name_Time, day=val_Time, P1_VL=val_VL, P1_CD4=val_CD4)

Row,name,day,P1_VL,P1_CD4
,String,Float64,Float64,Float64
1,Pre-infection,0.0,0.0,1200.0
2,0 Weeks,7.0,5.89,890.0
3,1 Weeks,14.0,7.96,600.0
4,2 Weeks,21.0,6.3,420.0
5,3 Weeks,28.0,5.91,550.0
6,4 Weeks,35.0,5.87,520.0
7,6 Months,180.0,4.5,560.0
8,1 Year,365.0,5.0,580.0


In [7]:
len = length(val_VL)

8

loss_function for nn

In [ ]:
depvars = [:T_1,:Ts_1,:T_2,:Ts_2,:V_1,:V_2,:E_1,:E_2]
function additional_loss(phi, θ , p)
    return sum(sum(abs2, phi[i](t_ , θ[depvars[i]]) .- u_[[i], :])/len for i in 1:1:8)
end

layer

In [ ]:
input_ = length(domains)
n = 8
chain1 = Lux.Chain(Dense(input_,n,Lux.σ),Dense(n,n,Lux.σ),Dense(n,n,Lux.σ),Dense(n,1))
chain2 = Lux.Chain(Dense(input_,n,Lux.σ),Dense(n,n,Lux.σ),Dense(n,n,Lux.σ),Dense(n,1))
chain3 = Lux.Chain(Dense(input_,n,Lux.σ),Dense(n,n,Lux.σ),Dense(n,n,Lux.σ),Dense(n,1))

In [ ]:
pam = [σ_, ρ, β]
var = [x(t), y(t), z(t)]

In [ ]:
var = [T_1(t),Ts_1(t),T_2(t),Ts_2(t),V_1(t),V_2(t),E_1(t),E_2(t)]

discretization

In [ ]:
discretization = NeuralPDE.PhysicsInformedNN([chain1 , chain2, chain3],NeuralPDE.GridTraining(dt), param_estim=true, additional_loss=additional_loss)
@named pde_system = PDESystem(eqs,bcs,domains,[t],var,pam, defaults=Dict([p .=> 1.0 for p in pam]))
prob = NeuralPDE.discretize(pde_system,discretization)
callback = function (p,l)
    println("Current loss is: $l")
    return false
end

In [ ]:
res = Optimization.solve(prob, BFGS(); callback = callback, maxiters=5000)

In [ ]:
p_ = res.u[end-2:end]

approximate

In [ ]:
minimizers = [res.u.depvar[depvars[i]] for i in 1:3]
ts = [infimum(d.domain):dt/10:supremum(d.domain) for d in domains][1]
u_predict  = [[discretization.phi[i]([t],minimizers[i])[1] for t in ts] for i in 1:3]
plot(sol)
plot!(ts, u_predict, label = ["x(t)" "y(t)" "z(t)"])